In [ ]:
pip install tensorflow keras-tuner scikit-learn seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf

In [ ]:
df = sns.load_dataset('iris')
df.head()
x = df.drop(columns=['sepal_length'])
y = df['sepal_length']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
le = LabelEncoder()
x_train['species'] = le.fit_transform(x_train['species'])
x_test['species']= le.transform(x_test['species'])


def test_model(hp):
    model=Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(hp.Int('unit1',min_value=32,max_value=128),activation=hp.Choice('activation1',['relu','tanh','sigmoid'])))
    model.add(Dense(hp.Int('unit2',min_value=32,max_value=128),activation=hp.Choice('activation2',['relu','tanh','sigmoid'])))
    model.add(Dense(1))
    model.compile(optimizer=SGD(hp.Float('optimize',min_value=0.00001,max_value=0.001),momentum=hp.Float('momentum',min_value=0.0,max_value=0.9)),loss='mean_squared_error',metrics=['mae'])
    return model


In [ ]:
tuner = kt.RandomSearch(
    test_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='test',
    project_name='iris_reg3'
)


tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, mae = best_model.evaluate(x_test, y_test)
print(f"Test MAE: {mae}")


Trial 10 Complete [00h 00m 04s]
val_mae: 0.4547576308250427

Best val_mae So Far: 0.29159554839134216
Total elapsed time: 00h 00m 32s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 40.2591 - mae: 6.2349 - val_loss: 19.9595 - val_mae: 4.3985
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 14.4180 - mae: 3.7018 - val_loss: 5.3691 - val_mae: 2.2621
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 3.7063 - mae: 1.8146 - val_loss: 1.2123 - val_mae: 1.0182
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6557 - mae: 0.7005 - val_loss: 0.3203 - val_mae: 0.4250
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2452 - mae: 0.3989 - val_loss: 0.1883 - val_mae: 0.3032
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1814 - mae: 0.3334 - val_loss: 0.1723 - val_mae: 0.2988
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1723 - mae: 0.3273 - val_loss: 0.1714 - val_mae: 0.2987
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0